# Sklearn
Основа материала взята из репозитория https://github.com/emeli-dral/sklearn_intro

Обновлено Константином Чукреевым

## sklearn.linear_model

**linear_model:**
* RidgeClassifier
* SGDClassifier
* SGDRegressor
* LinearRegression
* LogisticRegression
* Lasso
* etc

документация: http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model

примеры: http://scikit-learn.org/stable/modules/linear_model.html#linear-model

In [ ]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import seaborn as sns
import pandas as pd
sns.set();

In [ ]:
%pylab inline

In [ ]:
np.meshgrid?

In [ ]:
xx.ravel?

In [ ]:
def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

### Генерация данных

In [ ]:
datasets.make_blobs?

In [ ]:
blobs = datasets.make_blobs(centers=2, cluster_std=3.5, random_state=1)

In [ ]:
blobs[0].shape, blobs[1].shape

In [ ]:
tbl = pd.DataFrame(blobs[0])
tbl['target'] = blobs[1]

In [ ]:
tbl.head()

In [ ]:
sns.scatterplot(x=0, y=1, hue='target', data=tbl, s=70);

In [ ]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(blobs[0],
                                                                                    blobs[1], 
                                                                                    test_size=0.3,
                                                                                    random_state=1)

### Линейная классификация

#### RidgeClassifier

In [ ]:
#создание объекта - классификатора
ridge_classifier = linear_model.RidgeClassifier(random_state=1)

In [ ]:
#обучение классификатора
ridge_classifier.fit(train_data, train_labels)

In [ ]:
#применение обученного классификатора
ridge_predictions = ridge_classifier.predict(test_data)

In [ ]:
print(test_labels)

In [ ]:
print(ridge_predictions)

In [ ]:
#оценка качества классификации
metrics.accuracy_score(test_labels, ridge_predictions)

In [ ]:
print('Intercept', ridge_classifier.intercept_ )
print('Coefficients', ridge_classifier.coef_)

####  Нарисуем границу принятия решения нашим классификатором

In [ ]:
sns.scatterplot?

In [ ]:
xx, yy = make_meshgrid(blobs[0][:, 0], blobs[0][:, 1])
fig, sub = plt.subplots()
plot_contours(sub, ridge_classifier, xx, yy)
sns.scatterplot(x=0, y=1, hue='target', data=tbl, s=100);

#### LogisticRegression

In [ ]:
log_regressor = linear_model.LogisticRegression(random_state=0)

In [ ]:
log_regressor.fit(train_data, train_labels)

In [ ]:
lr_predictions = log_regressor.predict(test_data)

In [ ]:
lr_proba_predictions = log_regressor.predict_proba(test_data)

In [ ]:
print(test_labels)

In [ ]:
print(lr_predictions)

In [ ]:
print(lr_proba_predictions)

In [ ]:
print(metrics.roc_auc_score(test_labels, lr_predictions))

In [ ]:
metrics.roc_auc_score(test_labels, lr_proba_predictions[:,1]) == 1.0

In [ ]:
print(metrics.roc_auc_score(test_labels, lr_proba_predictions[:,1]))

In [ ]:
print(metrics.accuracy_score(test_labels, lr_predictions))

In [ ]:
print(metrics.accuracy_score(test_labels, ridge_predictions))

In [ ]:
print('Intercept', log_regressor.intercept_ )
print('Coefficient', log_regressor.coef_)

####  Нарисуем границу принятия решения нашим классификатором

In [ ]:
xx, yy = make_meshgrid(blobs[0][:, 0], blobs[0][:, 1])
fig, sub = plt.subplots()
plot_contours(sub, log_regressor, xx, yy)
sns.scatterplot(x=0, y=1, hue='target', data=tbl, s=100);

### Оценка качества по cross-validation

#### cross_val_score

In [ ]:
ridge_scoring = model_selection.cross_val_score(ridge_classifier,
                                                blobs[0],
                                                blobs[1],
                                                scoring='accuracy',
                                                cv=10)

In [ ]:
lr_scoring = model_selection.cross_val_score(log_regressor,
                                             blobs[0],
                                             blobs[1],
                                             scoring='accuracy',
                                             cv=10)

In [ ]:
ridge_scoring

In [ ]:
lr_scoring

In [ ]:
print('Ridge mean:{}, max:{}, min:{}, std:{:2f}'.format(ridge_scoring.mean(),
                                                        ridge_scoring.max(),
                                                        ridge_scoring.min(),
                                                        ridge_scoring.std()))

In [ ]:
print('Log mean:{}, max:{}, min:{}, std:{}'.format(lr_scoring.mean(),
                                                   lr_scoring.max(), 
                                                   lr_scoring.min(),
                                                   lr_scoring.std()))

#### cross_val_score с заданными scorer и cv_strategy

In [ ]:
scorer = metrics.make_scorer(metrics.accuracy_score)

In [ ]:
cv_strategy = model_selection.StratifiedShuffleSplit(n_splits=20,
                                                     test_size=0.3,
                                                     random_state=2)

In [ ]:
ridge_scoring = model_selection.cross_val_score(ridge_classifier,
                                                blobs[0],
                                                blobs[1],
                                                scoring=scorer,
                                                cv=cv_strategy.split(X=blobs[0], y=blobs[1]))

In [ ]:
lr_scoring = model_selection.cross_val_score(log_regressor,
                                             blobs[0],
                                             blobs[1],
                                             scoring=scorer,
                                             cv=cv_strategy.split(X=blobs[0], y=blobs[1]))

In [ ]:
print('Ridge mean:{}, max:{}, min:{}, std:{}'.format(ridge_scoring.mean(),
                                                     ridge_scoring.max(), 
                                                     ridge_scoring.min(),
                                                     ridge_scoring.std()))

In [ ]:
print('Log mean:{}, max:{}, min:{}, std:{}'.format(lr_scoring.mean(),
                                                   lr_scoring.max(), 
                                                   lr_scoring.min(), 
                                                   lr_scoring.std()))